In [1]:
import pandas as pd
import re
import numpy as np
from konlpy import jvm
from konlpy.tag import Okt
import datetime

In [26]:
df = pd.read_csv("temp_db2.csv")

In [27]:
df

,p_code,title,url,contents,interest,start_age,end_age,apply_start,apply_end,parse school res
0,0,국가근로장학금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,안정적인 학업여건 조성과 취업역량 제고를 위한 장학금,학자금 지원,-,-,NaN,NaN,['대학교 재학']
1,1,"월드프렌즈(World Friends Korea, WFK) - 일반봉사단",https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,개발도상국에 대한 주요 무상원조사업의 하나로 개발도상국의 수요를 바탕으로 우리가 비...,해외진출,19,-,NaN,NaN,[]
2,2,청년구직활동지원금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,스스로 취업을 준비하는 청년에게 월 50만원씩 최대 6개월 간 지원,"교육훈련·체험·인턴,",18,34,2019-10-01,2019-10-20,"['고등학교 졸업', '대학교 졸업', '석사 졸업']"
3,3,청년취업성공패키지,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,단계별로 차근차근 취업에 성공하자! 상담-능력개발-취업알선 3단계에 걸쳐 취업을 지...,"교육훈련·체험·인턴,",18,34,NaN,NaN,[]
4,4,내일배움카드(실업자ㆍ구직자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,구직자에게 직업능력개발훈련 기회를 제공하여 기술･기능 습득을 지원,교육훈련·체험·인턴,15,-,NaN,NaN,[]
5,5,내일배움카드(근로자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"중소기업 근로자, 비정규직 근로자 등의 직업훈련기회 확대를 통한 평생 고용가능성 제고",교육훈련·체험·인턴,-,-,2018-07-01,NaN,[]
6,6,청년 우대형 청약통장,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"저소득, 무주택 청년을 위한 청약 통장",생활비 지원 및 금융 혜택,19,34,2018-07-31,2021-12-31,[]
7,7,청년 보증부 월세대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,신혼부부·청년에게 월세대출 연 1%대의 저금리 상품 제공,생활비 지원 및 금융 혜택,-,34,NaN,NaN,[]
8,8,중소기업취업청년 전월세보증금대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,중소기업에 취업한 청년들에게 저리로 중소기업 취업청년 전월세보증금을 지원,주거 지원,15,34,NaN,NaN,[]
9,9,스마트 벤처 캠퍼스,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"유망지식서비스(앱/웹, 콘텐츠, SW, 융합)분야 청년창업자 집중 지원",R&D 지원,-,39,NaN,NaN,[]


In [28]:
policy_table_query = """CREATE TABLE policy (
        p_code int,
        title varchar(50) ,
        uri varchar(150),
        apply_start timestamp,
        apply_end   timestamp,
        start_age   int,
        end_age     int,
        contents    varchar(10000),
        crawing_date  timestamp,
        expiration_flag tinyint(1),
        CONSTRAINT policy_pk PRIMARY KEY (p_code)
         
);"""


In [29]:
datetime.datetime.now()

datetime.datetime(2019, 10, 26, 15, 44, 30, 874977)

In [30]:
df_policy = df[['p_code','title','url','apply_start','apply_end','start_age','end_age','contents']]

In [31]:
df_policy['crawing_date'] = datetime.datetime.now()

C:\Users\moggi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_policy['expiration_flag'] = 0

In [33]:
df_policy = df_policy.rename(columns = {'url':'uri'} )

In [34]:
df['interest'].unique()

array(['학자금 지원', '해외진출', '교육훈련·체험·인턴,', '교육훈련·체험·인턴', '생활비 지원 및 금융 혜택 ',
       '주거 지원', 'R&D 지원', '경영 지원,', '중소기업 취업지원', '전문분야 취업지원',
       '전문분야 취업지원,', '-', '자본금 지원', '문화', '경영 지원', '건강,', '건강', 'R&D 지원,'],
      dtype=object)

In [35]:
def parse_interest(x):
    res = [0,0,0,0]
    if x['interest'] in ['교육훈련·체험·인턴','전문분야 취업지원','중소기업 취업지원','해외진출']:
        res[0] = 1
    elif x['interest'] in ['R&D 지원','경영 지원','자본금 지원']:
        res[1] = 1
    elif x['interest'] in ['건강','문화']:
        res[2] = 1
    elif x['interest'] in ['생활비 지원 및 금융 혜택','주거 지원','학자금 지원']:
        res[3] = 1
    
    x['Employment_sup'] = res[0]
    x['Startup_sup'] = res[1]
    x['Life_welfare'] = res[2]
    x['Residential_finance'] = res[3]
    
    return x

In [36]:
'학자금 지원' in ['교육훈련·체험·인턴','전문분야 취업지원','중소기업 취업지원','해외진출']

False

In [37]:
df = df.apply(parse_interest,axis=1)

In [38]:
df

,p_code,title,url,contents,interest,start_age,end_age,apply_start,apply_end,parse school res,Employment_sup,Startup_sup,Life_welfare,Residential_finance
0,0,국가근로장학금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,안정적인 학업여건 조성과 취업역량 제고를 위한 장학금,학자금 지원,-,-,NaN,NaN,['대학교 재학'],0,0,0,1
1,1,"월드프렌즈(World Friends Korea, WFK) - 일반봉사단",https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,개발도상국에 대한 주요 무상원조사업의 하나로 개발도상국의 수요를 바탕으로 우리가 비...,해외진출,19,-,NaN,NaN,[],1,0,0,0
2,2,청년구직활동지원금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,스스로 취업을 준비하는 청년에게 월 50만원씩 최대 6개월 간 지원,"교육훈련·체험·인턴,",18,34,2019-10-01,2019-10-20,"['고등학교 졸업', '대학교 졸업', '석사 졸업']",0,0,0,0
3,3,청년취업성공패키지,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,단계별로 차근차근 취업에 성공하자! 상담-능력개발-취업알선 3단계에 걸쳐 취업을 지...,"교육훈련·체험·인턴,",18,34,NaN,NaN,[],0,0,0,0
4,4,내일배움카드(실업자ㆍ구직자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,구직자에게 직업능력개발훈련 기회를 제공하여 기술･기능 습득을 지원,교육훈련·체험·인턴,15,-,NaN,NaN,[],1,0,0,0
5,5,내일배움카드(근로자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"중소기업 근로자, 비정규직 근로자 등의 직업훈련기회 확대를 통한 평생 고용가능성 제고",교육훈련·체험·인턴,-,-,2018-07-01,NaN,[],1,0,0,0
6,6,청년 우대형 청약통장,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"저소득, 무주택 청년을 위한 청약 통장",생활비 지원 및 금융 혜택,19,34,2018-07-31,2021-12-31,[],0,0,0,0
7,7,청년 보증부 월세대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,신혼부부·청년에게 월세대출 연 1%대의 저금리 상품 제공,생활비 지원 및 금융 혜택,-,34,NaN,NaN,[],0,0,0,0
8,8,중소기업취업청년 전월세보증금대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,중소기업에 취업한 청년들에게 저리로 중소기업 취업청년 전월세보증금을 지원,주거 지원,15,34,NaN,NaN,[],0,0,0,1
9,9,스마트 벤처 캠퍼스,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,"유망지식서비스(앱/웹, 콘텐츠, SW, 융합)분야 청년창업자 집중 지원",R&D 지원,-,39,NaN,NaN,[],0,1,0,0


In [40]:
df_policy.to_csv('temp_db_make_table.csv')

In [41]:
df_interest = df[['p_code','Employment_sup','Startup_sup','Life_welfare','Residential_finance']]

In [43]:
df_interest.to_csv('temp_db_interest.csv')

In [47]:
df_policy['start_age'] = df_policy['start_age'].apply(lambda x: None if x == "-" else x)

In [48]:
df_policy['end_age'] = df_policy['end_age'].apply(lambda x: None if x == "-" else x)

In [49]:
df_policy

,p_code,title,uri,apply_start,apply_end,start_age,end_age,contents,crawing_date,expiration_flag
0,0,국가근로장학금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,None,None,안정적인 학업여건 조성과 취업역량 제고를 위한 장학금,2019-10-26 15:44:31.912379,0
1,1,"월드프렌즈(World Friends Korea, WFK) - 일반봉사단",https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,19,None,개발도상국에 대한 주요 무상원조사업의 하나로 개발도상국의 수요를 바탕으로 우리가 비...,2019-10-26 15:44:31.912379,0
2,2,청년구직활동지원금,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,2019-10-01,2019-10-20,18,34,스스로 취업을 준비하는 청년에게 월 50만원씩 최대 6개월 간 지원,2019-10-26 15:44:31.912379,0
3,3,청년취업성공패키지,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,18,34,단계별로 차근차근 취업에 성공하자! 상담-능력개발-취업알선 3단계에 걸쳐 취업을 지...,2019-10-26 15:44:31.912379,0
4,4,내일배움카드(실업자ㆍ구직자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,15,None,구직자에게 직업능력개발훈련 기회를 제공하여 기술･기능 습득을 지원,2019-10-26 15:44:31.912379,0
5,5,내일배움카드(근로자),https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,2018-07-01,NaN,None,None,"중소기업 근로자, 비정규직 근로자 등의 직업훈련기회 확대를 통한 평생 고용가능성 제고",2019-10-26 15:44:31.912379,0
6,6,청년 우대형 청약통장,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,2018-07-31,2021-12-31,19,34,"저소득, 무주택 청년을 위한 청약 통장",2019-10-26 15:44:31.912379,0
7,7,청년 보증부 월세대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,None,34,신혼부부·청년에게 월세대출 연 1%대의 저금리 상품 제공,2019-10-26 15:44:31.912379,0
8,8,중소기업취업청년 전월세보증금대출,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,15,34,중소기업에 취업한 청년들에게 저리로 중소기업 취업청년 전월세보증금을 지원,2019-10-26 15:44:31.912379,0
9,9,스마트 벤처 캠퍼스,https://www.youthcenter.go.kr/jynEmpSptNew/jyn...,NaN,NaN,None,39,"유망지식서비스(앱/웹, 콘텐츠, SW, 융합)분야 청년창업자 집중 지원",2019-10-26 15:44:31.912379,0


In [50]:
df_policy.to_csv("temp_db_make_table.csv")

In [53]:
pattern_not_num = re.compile(r"[^0-9]+")

In [56]:
pattern_num.search("1s2")

<re.Match object; span=(1, 2), match='s'>

In [60]:
df_policy['start_age'] = df_policy['start_age'].apply(lambda x: None if pattern_num.search(str(x)) else x)

In [61]:
df_policy['end_age'] = df_policy['end_age'].apply(lambda x: None if pattern_num.search(str(x)) else x)

In [62]:
df_policy.to_csv("temp_db_make_table.csv")